# This first notebook is for pulling down the relevant data from the Synapse API, storing resulting table information in dataframes, and generating a list of file paths to downloaded accelerometer files that will be needed to access the data itself. Note that accessing the actual data requires access to the Synapse API.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [2]:
import synapseclient
import numpy as np
import pandas as pd

In [3]:
# Login into the Synapse Client

syn = synapseclient.Synapse()

# Enter login and password
#syn.login('login','password')

# Query demographics table

In [14]:
demographicstable = syn.tableQuery('select * from syn5511429')

# Query iPhone6 and iPhone6Plus accelerometer data

In [15]:
walkingtable = syn.tableQuery("select * from syn5511449 where phoneInfo = 'iPhone 6' or phoneInfo = 'iPhone 6 Plus'")
walking_df = walkingtable.asDataFrame()


walking_df = walking_df.drop(['createdOn', 'appVersion', 'phoneInfo','accel_walking_outbound.json.items',
                              'pedometer_walking_outbound.json.items','accel_walking_return.json.items',
                              'deviceMotion_walking_return.json.items','pedometer_walking_return.json.items',
                              'accel_walking_rest.json.items'],axis=1)

walking_df = walking_df.dropna()

filePaths_DMoutbound = syn.downloadTableColumns(walkingtable, ['deviceMotion_walking_outbound.json.items'])
filePaths_DMrest = syn.downloadTableColumns(walkingtable, ['deviceMotion_walking_rest.json.items'])

# Store in dataframes

In [16]:
demographics_df=demographicstable.asDataFrame()

wlist = np.arange(len(walking_df))

DMoutboundPathlist=[filePaths_DMoutbound[str(int(walking_df.ix[entry,'deviceMotion_walking_outbound.json.items']))] for entry in wlist] 
DMrestPathlist = [filePaths_DMrest[str(int(walking_df.ix[entry,'deviceMotion_walking_rest.json.items']))] for entry in wlist]

walking_df['DMoutboundPaths'] = DMoutboundPathlist
walking_df['DMrestPaths'] = DMrestPathlist

# Pickle the dataframes and filepath lists

In [17]:
demographics_df.to_pickle('demographics_df.pkl')
walking_df.to_pickle('walking_df.pkl')